# DVC Options - Metrics and Plots
## The DVC way

In the following part we will look into metrics and plots from ZnTrack Nodes.
All `dvc run` options listed [here](https://dvc.org/doc/command-reference/run#options) can be used via `dvc.<option>`.
With the exception of params, which is handled automatically.
All these options take either `str` or `pathlib.Path` directed to the file the content should be stored in.
As shown before, `dvc.deps` can also take another `Node` as an argument.

In [1]:
from zntrack import Node, dvc, zn, config
from pathlib import Path
import json
import pandas as pd
import numpy as np

In [2]:
config.nb_name = "04_metrics_and_plots.ipynb"

In [3]:
from zntrack.utils import cwd_temp_dir

temp_dir = cwd_temp_dir()

In [4]:
!git init
!dvc init

Initialized empty Git repository in C:/Users/fabia/AppData/Local/Temp/tmpob9qmb6b/.git/
Initialized DVC repository.

You can now commit the changes to git.

+---------------------------------------------------------------------+
|                                                                     |
|        DVC has enabled anonymous aggregate usage analytics.         |
|     Read the analytics documentation (and how to opt-out) here:     |
|             <https://dvc.org/doc/user-guide/analytics>              |
|                                                                     |
+---------------------------------------------------------------------+

What's next?
------------
- Check out the documentation: <https://dvc.org/doc>
- Get help and share ideas: <https://dvc.org/chat>
- Star us on GitHub: <https://github.com/iterative/dvc>


In the following we define a simple Node that produces a metric and a plot output using `json` and `pandas`.
We will queue multiple experiments with different outputs and then compare them afterwards.
With `@Node(silent=True)` we can reduce the amount of logs that will be displayed.

In [5]:
class MetricAndPlot(Node):
    my_metric: Path = dvc.metrics(Path("my_metric.json"))
    my_plots: Path = dvc.plots("my_plots.csv")
    pre_factor = zn.params()

    def __init__(self, pre_factor=1.0, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.pre_factor = pre_factor

    def run(self):
        self.my_metric.write_text(
            json.dumps(
                {"metric_1": 17 * self.pre_factor, "metric_2": 42 * self.pre_factor}
            )
        )

        x_data = np.linspace(0, 1.0 * self.pre_factor, 1000)
        y_data = np.exp(x_data)
        df = pd.DataFrame({"y": y_data, "x": x_data}).set_index("x")

        df.to_csv(self.my_plots)

In [6]:
MetricAndPlot().write_graph(silent=True)
!dvc repro
!git add .
!git commit -m "First Run"

2022-01-13 20:32:53,215 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 20:32:53,216 (WARNING): Converting 04_metrics_and_plots.ipynb to file MetricAndPlot.py
Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot.load(name='MetricAndPlot').run_and_save()" 
Generating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock
Use `dvc push` to send your updates to remote storage.
[master (root-commit) 07f23d2] First Run
 17 files changed, 1054 insertions(+)
 create mode 100644 .dvc/.gitignore
 create mode 100644 .dvc/config
 create mode 100644 .dvc/plots/confusion.json
 create mode 100644 .dvc/plots/confusion_normalized.json
 create mode 100644 .dvc/plots/linear.json
 create mode 100644 .dvc/plots/scatter.json
 create mode 100644 .dvc/plots/simple.json
 create mode 1

In [7]:
MetricAndPlot(pre_factor=2).write_graph(silent=True)
!dvc exp run --queue --name "factor_2"
MetricAndPlot(pre_factor=3).write_graph(silent=True)
!dvc exp run --queue --name "factor_3"
MetricAndPlot(pre_factor=4).write_graph(silent=True)
!dvc exp run --queue --name "factor_4"
MetricAndPlot(pre_factor=5).write_graph(silent=True)
!dvc exp run --queue --name "factor_5"

2022-01-13 20:33:00,287 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 20:33:00,287 (WARNING): Converting 04_metrics_and_plots.ipynb to file MetricAndPlot.py
Queued experiment '48e82ed' for future execution.
2022-01-13 20:33:05,789 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 20:33:05,789 (WARNING): Converting 04_metrics_and_plots.ipynb to file MetricAndPlot.py
Queued experiment 'e20fbbc' for future execution.
2022-01-13 20:33:11,085 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 20:33:11,085 (WARNING): Converting 04_metrics_and_plots.ipynb to file MetricAndPlot.py
Queued experiment '31d08c0' for future exe

In [8]:
!dvc exp run --run-all -j 4

Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot.load(name='MetricAndPlot').run_and_save()" 
Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot.load(name='MetricAndPlot').run_and_save()" 
Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot.load(name='MetricAndPlot').run_and_save()" 
Running stage 'MetricAndPlot':
> python -c "from src.MetricAndPlot import MetricAndPlot; MetricAndPlot.load(name='MetricAndPlot').run_and_save()" 
Updating lock file 'dvc.lock'
Updating lock file 'dvc.lock'
Updating lock file 'dvc.lock'
Updating lock file 'dvc.lock'

Reproduced experiment(s): factor_5, factor_4, factor_3, factor_2
To apply the results of an experiment to your workspace run:

	dvc exp apply <exp>

To promote an experiment to a Git branch run:

	dvc exp branch <exp> <branch>



Now that all experiments are done, we can look at the metrics directly with `dvc exp show` or `dvc metrics show/diff`

In [9]:
!dvc exp show --csv --no-timestamp > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

,rev,typ,parent,metric_1,metric_2,MetricAndPlot.pre_factor
Experiment,,,,,,
NaN,workspace,baseline,NaN,17.0,42.0,5.0
master,07f23d2,baseline,NaN,17.0,42.0,1.0
factor_2,8b85c5e,branch_commit,NaN,34.0,84.0,2.0
factor_3,96ea8c5,branch_commit,NaN,51.0,126.0,3.0
factor_4,76141a8,branch_commit,NaN,68.0,168.0,4.0
factor_5,fc0bc79,branch_base,NaN,85.0,210.0,5.0


We can also use `dvc plots show/diff` to evaluate the plot data that we produced.

In [10]:
!dvc plots diff HEAD factor_2 factor_3 factor_4 factor_5

file:///C:/Users/fabia/AppData/Local/Temp/tmpob9qmb6b/dvc_plots/index.html


In [11]:
from IPython.display import IFrame

In [12]:
IFrame(src="../../_static/plots/overview_04_plots.html", width=700, height=600)

## The ZnTrack way

ZnTrack provides and easier way to handle metrics. Similar to `zn.outs()` which does not require defining a path to outs file, one can use `zn.metrics`.

In [13]:
class ZnTrackMetric(Node):
    my_metric = zn.metrics()

    def run(self):
        self.my_metric = {"alpha": 1.0, "beta": 0.00473}


ZnTrackMetric().write_graph(no_exec=False)

2022-01-13 20:33:30,845 (WARNING): Jupyter support is an experimental feature! Please save your notebook before running this command!
Submit issues to https://github.com/zincware/ZnTrack.
2022-01-13 20:33:30,847 (WARNING): Converting 04_metrics_and_plots.ipynb to file ZnTrackMetric.py
2022-01-13 20:33:33,275 (WARNING): --- Writing new DVC file! ---
2022-01-13 20:33:33,276 (WARNING): You will not be able to see the stdout/stderr of the process in real time!
2022-01-13 20:33:36,076 (INFO): Running stage 'ZnTrackMetric':
> python -c "from src.ZnTrackMetric import ZnTrackMetric; ZnTrackMetric.load(name='ZnTrackMetric').run_and_save()" 
Adding stage 'ZnTrackMetric' in 'dvc.yaml'
Updating lock file 'dvc.lock'

To track the changes with git, run:

	git add dvc.lock dvc.yaml



In [14]:
!dvc exp show --csv --no-timestamp > exp_show.csv
pd.read_csv("exp_show.csv", index_col=0)

,rev,typ,parent,metric_1,metric_2,my_metric.alpha,my_metric.beta,MetricAndPlot.pre_factor
Experiment,,,,,,,,
NaN,workspace,baseline,NaN,17.0,42.0,1.0,0.00473,5.0
master,07f23d2,baseline,NaN,17.0,42.0,1.0,NaN,NaN
factor_2,8b85c5e,branch_commit,NaN,34.0,84.0,2.0,NaN,NaN
factor_3,96ea8c5,branch_commit,NaN,51.0,126.0,3.0,NaN,NaN
factor_5,fc0bc79,branch_commit,NaN,85.0,210.0,5.0,NaN,NaN
factor_4,76141a8,branch_base,NaN,68.0,168.0,4.0,NaN,NaN


In [ ]:
temp_dir.cleanup()